In [ ]:
# 5 Major parts

![Alt text](image-12.png)

In [3]:
import apache_beam as beam

In [ ]:
#  p1 = beam.Pipeline()#pipeline method controls lifecycle

In [ ]:
# attendance_count(
#     p1
#      |


# )

In [ ]:
#Read Transforms supports below files
#text avro parquet

![Alt text](image-13.png)

![Alt text](image-14.png)

![Alt text](image-15.png)

![Alt text](image-18.png)

In [ ]:
# Python only support pubsub

![Alt text](image-19.png)

![Alt text](image-20.png)

In [ ]:
#create transform to create own data

In [6]:
p2 = beam.Pipeline()

lines =(
    p2
    |beam.Create([1,2,3,4,5,6,7])
    |beam.io.WriteToText('data/outcreate1')


)
p2.run()

In [7]:
#Visualize the output
!{('head -n 20 data/outcreate1-00000-of-00001')}

1
2
3
4
5
6
7


In [ ]:
#Write Transforms

![Alt text](image-21.png)

![Alt text](image-22.png)

![Alt text](image-23.png)

![Alt text](image-24.png)

In [ ]:
#Pubsub

![Alt text](image-25.png)

In [14]:
p1 = beam.Pipeline()
def SplitRow(element):
    return element.split(',')

attendance_count=(
    p1
     |beam.io.ReadFromText("dept_data.txt")
     |beam.Map(SplitRow) #takes one element as input and output
     |beam.Filter(lambda record : record[3] == 'Accounts') 
     |beam.Map(lambda record : (record[1],1)) # append 1 to each record and get the name and 1 as two columns
     |beam.CombinePerKey(sum)
     |beam.io.WriteToText('data/output_new')


)

p1.run()
!{('head -n 20 data/output_new-00000-of-00001')}


('Marco', 31)
('Rebekah', 31)
('Itoe', 31)
('Edouard', 31)
('Kyle', 62)
('Kumiko', 31)
('Gaston', 31)
('Ayumi', 30)


In [ ]:
#Map vs Flatmap
# Flat map 1 to Many mapping -> multiple output possible
# Map only 1 to 1 mapping -> single output

In [15]:

#labelling
#other ways of running pipeline

def SplitRow(element):
    return element.split(',')

with beam.Pipeline() as p1:
    attendance_count=(
        p1
        |'Read from file' >> beam.io.ReadFromText("dept_data.txt")
        |'Map' >> beam.Map(SplitRow) #takes one element as input and output
        |beam.Filter(lambda record : record[3] == 'Accounts') 
        |beam.Map(lambda record : (record[1],1)) # append 1 to each record and get the name and 1 as two columns
        |beam.CombinePerKey(sum)
        |beam.io.WriteToText('data/output_new')


    )

#p1.run()
!{('head -n 20 data/output_new-00000-of-00001')}


('Marco', 31)
('Rebekah', 31)
('Itoe', 31)
('Edouard', 31)
('Kyle', 62)
('Kumiko', 31)
('Gaston', 31)
('Ayumi', 30)


In [ ]:
# Branching and flattening
# flattens combines two pcollection if they have same datatype and columns

![Alt text](image-26.png)

In [ ]:
# Pardo is generic case. Map and Flatmap are special cases of pardo transform

# Create a class inherating DoFn class. DoFn implements distributed processing
# We need to overwrite only the process method.

#can also use lambda function. beam internally creates dofn lightweight objects

![Alt text](image-27.png)

import apache_beam as beam

class SplitRow(beam.DoFn):
  
  def process(self, element):
    # return type -> list
    return  [element.split(',')]
  

class FilterAccountsEmployee(beam.DoFn):
  
  def process(self, element):
    if element[3] == 'Accounts':
      return [element]  
    
class PairEmployees(beam.DoFn):
  
  def process(self, element):
    return [(element[3]+","+element[1], 1)]    
  
class Counting(beam.DoFn):
  
  def process(self, element):
    # return type -> list
    (key, values) = element           # [Marco, Accounts  [1,1,1,1....] , Rebekah, Accounts [1,1,1,1,....] ]
    return [(key, sum(values))]
     

p1 = beam.Pipeline()

attendance_count = (
    
   p1
    |beam.io.ReadFromText('dept_data.txt')
    
    |beam.ParDo(SplitRow())
   # | 'Compute WordLength' >> beam.ParDo(lambda element: [ element.split(',') ]) 

    |beam.ParDo(FilterAccountsEmployee())
    |beam.ParDo(PairEmployees())
    | 'Group ' >> beam.GroupByKey()
    | 'Sum using ParDo' >> beam.ParDo(Counting())  
    
    |beam.io.WriteToText('data/output_new_final')
  
)

p1.run()

# Sample the first 20 results, remember there are no ordering guarantees.
!{('head -n 20 data/output_new_final-00000-of-00001')}